In [2]:
import numpy as np
from Convolutional import *
from Classification import *
import matplotlib.pyplot as plt
"""
TO DO

Standartization
Batch
Batch Normalization
Weight-Initilazition Trick



"""

In [75]:
def C_backward_model(dX, caches):
    #it needs to check to design
    grads={}
    L=len(caches) # number of layers in the network
    convolutional_cache ,pooling_cache, shape = caches[0]
    grads["dA"+str(L+1)]=dX.reshape(shape)
    for l in reversed(range(L)):
        
        convolutional_cache ,pooling_cache, shape = caches[l]
        
        X = pool_backward(grads["dA"+str(l+2)], pooling_cache)
        
        grads["dA" + str(l+1)], grads["dW" + str(l+1)], grads["db" + str(l+1)]  = conv_backward(X,convolutional_cache)
   
    return grads
    
def C_forward_model(X, Cparameters, hparameters):
        
    caches=[]
    # for this code pool forward and conv_forward are in same layer so 2 layer is actually 1 layer
    
    L = len(Cparameters) // 2 # number of layers in the network
    
    for j in range(0,L):
        X, convolutional_cache = conv_forward(X,Cparameters['W' + str(j+1)],Cparameters['b' + str(j+1)],hparameters[j][0])
        
        X, pooling_cache = pool_forward(X,hparameters[j][1])
        caches.append([convolutional_cache,pooling_cache,X.shape])
    
    X = X.reshape((X.shape[0] , np.prod(X.shape[1:]))) #[[[1,2],[3,4]],[[5,6],[7,8]]] -> [[1,2,3,4],[5,6,7,8]]
    
    return X, caches

In [102]:

def L_layer_model(X, Y,conv_layers_dims, layers_dims, hparameters,learning_rate = 0.0075, num_iterations = 300, print_cost=True):

    costs = []    #keep track of cost
    
    Cparameters, parameters = initialize_parameters_deep(conv_layers_dims,layers_dims)
    #Standardize
    
    for i in range(0, num_iterations):
        
        #Forward Convolutional
        Z, Ccaches= C_forward_model(X,Cparameters,hparameters)
        
        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(Z.T, parameters)
        
        #Compute cost
        cost = compute_cost(AL, Y)
    
        #Backward propagation
        grads = L_model_backward(AL, Y, caches)
        
        Cgrads = C_backward_model(grads["dA1"], Ccaches)
        
        #Update parameters
        Cparameters, parameters = update_parameters(Cparameters, parameters, Cgrads, grads, learning_rate)
        
        #Print the cost every 100 training example
        if print_cost and i % 1 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            costs.append(cost)
            
    #plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return Cparameters, parameters
    
def initialize_parameters_deep(conv_layer_dims,layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(1)
    
    #Conv
    
    Cparameters = {}
    L = len(conv_layer_dims) 
    for l in range(1, L+1):
        Cparameters['W' + str(l)] = np.random.randn(conv_layer_dims[l-1][0],conv_layer_dims[l-1][1]
                                                    ,conv_layer_dims[l-1][2],conv_layer_dims[l-1][3]) 
        Cparameters['b' + str(l)] = np.zeros((1,1,1,conv_layer_dims[l-1][3]))
        assert(Cparameters['W' + str(l)].shape == (conv_layer_dims[l-1][0],conv_layer_dims[l-1][1]
                                                    ,conv_layer_dims[l-1][2],conv_layer_dims[l-1][3]))
        assert(Cparameters['b' + str(l)].shape == (1,1,1,conv_layer_dims[l-1][3]))
    
    #Dense
    
    parameters = {}
    L = len(layer_dims)           # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return Cparameters, parameters



def update_parameters(Cparameters,parameters, Cgrads, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    L = len(Cparameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        Cparameters["W" + str(l+1)] = Cparameters["W" + str(l+1)] - learning_rate * Cgrads["dW" + str(l+1)]
        Cparameters["b" + str(l+1)] = Cparameters["b" + str(l+1)] - learning_rate * Cgrads["db" + str(l+1)]
    
            
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return Cparameters, parameters


In [103]:
A=np.random.randint(255,size=(2,100,100,3))
#A.shape
A

array([[[[108, 196, 215],
         [134, 220, 202],
         [ 89, 100, 221],
         ...,
         [164, 223,  47],
         [156,  55, 106],
         [ 86, 116, 169]],

        [[100, 133, 185],
         [ 50, 187,   2],
         [ 40, 131, 113],
         ...,
         [225, 111,  40],
         [ 29, 146, 202],
         [220, 204, 166]],

        [[200, 115, 178],
         [204,  98,  71],
         [176, 247, 188],
         ...,
         [124, 101, 142],
         [241,  60,  73],
         [172, 182, 174]],

        ...,

        [[ 44, 254,  13],
         [179, 174,  37],
         [216, 243,  27],
         ...,
         [ 15,  98,   1],
         [189, 222, 131],
         [ 30, 148, 131]],

        [[ 69, 166, 121],
         [250,  31, 154],
         [243,  62,  87],
         ...,
         [ 76, 187, 227],
         [ 30,  69, 132],
         [220, 101, 217]],

        [[ 92,  63,   4],
         [ 88,  51,  61],
         [ 41, 178, 254],
         ...,
         [172,  20, 242],
        

In [104]:
hparameters = [[{"pad" : 2,"stride": 1},{"f" : 2,"stride": 1}]]

In [ ]:
Cp,p = L_layer_model(A,np.zeros((1,2)),[[2,2,3,8]],[83232,3,1],hparameters)


Cost after iteration 0: 0.000165
Cost after iteration 1: 0.000000
